<a href="https://colab.research.google.com/github/rishav-eulb/ai-tools/blob/rishav-eulb-patch-3/src/asr/MMS_ASR_Inference_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Running MMS-ASR inference in Colab

In this notebook, we will give an example on how to run simple ASR inference using MMS ASR model. 

Credit to epk2112 [(github)](https://github.com/epk2112/fairseq_meta_mms_Google_Colab_implementation)

## Step 1: Clone fairseq-py and install latest version

In [ ]:
!mkdir "temp_dir"
!git clone https://github.com/pytorch/fairseq

# Change current working directory
!pwd
%cd "/content/fairseq"
!pip install --editable ./ 
!pip install tensorboardX


Cloning into 'fairseq'...
remote: Enumerating objects: 34697, done.
remote: Counting objects: 100% (154/154), done.
remote: Compressing objects: 100% (96/96), done.
remote: Total 34697 (delta 73), reused 124 (delta 53), pack-reused 34543
Receiving objects: 100% (34697/34697), 24.76 MiB | 5.98 MiB/s, done.
Resolving deltas: 100% (25174/25174), done.
/content
/content/fairseq
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Obtaining file:///content/fairseq
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.8/123.8 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.9/118.9 kB 16.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 272.7/272.7 kB 26.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

## 2. Download MMS model
Un-comment to download your preferred model.
In this example, we use MMS-FL102 for demo purposes.
For better model quality and language coverage, user can use MMS-1B-ALL model instead (but it would require more RAM, so please use Colab-Pro instead of Colab-Free).


In [ ]:
# MMS-1B:FL102 model - 102 Languages - FLEURS Dataset
!wget -P ./models_new 'https://dl.fbaipublicfiles.com/mms/asr/mms1b_fl102.pt'

# # MMS-1B:L1107 - 1107 Languages - MMS-lab Dataset
# !wget -P ./models_new 'https://dl.fbaipublicfiles.com/mms/asr/mms1b_l1107.pt'

# # MMS-1B-all - 1162 Languages - MMS-lab + FLEURS + CV + VP + MLS
# !wget -P ./models_new 'https://dl.fbaipublicfiles.com/mms/asr/mms1b_all.pt'

--2023-05-29 00:53:31--  https://dl.fbaipublicfiles.com/mms/asr/mms1b_fl102.pt
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 13.227.219.10, 13.227.219.70, 13.227.219.33, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|13.227.219.10|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4851043301 (4.5G) [binary/octet-stream]
Saving to: ‘./models_new/mms1b_fl102.pt’

mms1b_fl102.pt      100%[===================>]   4.52G  84.1MB/s    in 24s     

2023-05-29 00:53:56 (190 MB/s) - ‘./models_new/mms1b_fl102.pt’ saved [4851043301/4851043301]



## 3. Prepare audio file
Create a folder on path '/content/audio_samples/' and upload your .wav audio files that you need to transcribe e.g. '/content/audio_samples/audio.wav' 

Note: You need to make sure that the audio data you are using has a sample rate of 16kHz You can easily do this with FFMPEG like the example below that converts .mp3 file to .wav and fixing the audio sample rate

Here, we use a FLEURS english MP3 audio for the example.

In [ ]:
!wget -P /content/audio_samples 
!ffmpeg -y -i /content/audio_samples/audio_18.mp3 -ar 16000 /content/audio_samples/audio_18.wav

wget: missing URL
Usage: wget [OPTION]... [URL]...

Try `wget --help' for more options.
ffmpeg version 4.2.7-0ubuntu0.1 Copyright (c) 2000-2022 the FFmpeg developers
  built with gcc 9 (Ubuntu 9.4.0-1ubuntu1~20.04.1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-avresample --disable-filter=resample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librsvg --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-

# 4: Run Inference and transcribe your audio(s)


In the below example, we will transcribe a sentence in English.

To transcribe other languages: 
1. Go to [MMS README ASR section](https://github.com/facebookresearch/fairseq/tree/main/examples/mms#asr)
2. Open Supported languages link
3. Find your target languages based on Language Name column
4. Copy the corresponding Iso Code
5. Replace `--lang "eng"` with new Iso Code

To improve the transcription quality, user can use language-model (LM) decoding by following this instruction [ASR LM decoding](https://github.com/facebookresearch/fairseq/tree/main/examples/mms#asr)

In [ ]:
import os

os.environ["TMPDIR"] = '/content/temp_dir'
os.environ["PYTHONPATH"] = "."
os.environ["PREFIX"] = "INFER"
os.environ["HYDRA_FULL_ERROR"] = "1"
os.environ["USER"] = "micro"

!python examples/mms/asr/infer/mms_infer.py --model "/content/fairseq/models_new/mms1b_fl102.pt" --lang "ory" --audio "/content/audio_samples/audio_18.wav"


In [ ]:
!pip install -U nltk


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import nltk
from nltk.translate.bleu_score import sentence_bleu

# Tokenize candidate and reference sentences
reference = "ନମସ୍କାର। ଓଡିଶା ସରକାରଙ୍କ ପ୍ରାଣୀ ଚିକିତ୍ସା ଆଉ ପ୍ରାଣୀ ପାଳନ ସୂଚନା ସେବା କୁ ଆପଣଁକୁ ସ୍ଵାଗତ. ଚାଷ ଭାଇ ଓ ଭଉଣୀ ମାନେ ଆସନ୍ତୁ ଗାଈ ଓ ମଇଁଷି ମାନଙ୍କ କ୍ରୋଧାନ୍ବିତ ରୋଗ ର ନିରାକରଣ ବିଷୟ ରେ ଜାଣିବା. ନିୟମିତ ସ୍ବାସ୍ଥ୍ୟ ର ନିରୀକ୍ଷଣ,ଟୀକା କରଣ ଓ କୃମି ନାଶକ ଔଷଧ ର ପ୍ରୟୋଗ, Probiotics ଓ ଯକୃତ ବର୍ଦ୍ଧକ ର ବ୍ୟବହାର,ସଫା ପାଣି ଯୋଗାଇବା, ସୁଷମ ସନ୍ତୁଳିତ ଖାଦ୍ୟ ଇତ୍ୟାଦି ଦେବା, ରୋଷେଇ ପରେ ବଳି ପଡୁଥିବା ଖାଦ୍ୟ ଇତ୍ୟାଦି ନ ଦେବା,ପଶୁ ମାନଙ୍କ ଲାଗି ପ୍ରକୃଷ୍ଟ୍ ଖାଦ୍ୟ ଇତ୍ୟାଦି ଖାଇବାକୁ ଦେବା,ଫିମ୍ପି ଯୁକ୍ତ ଖାଦ୍ୟ ନ ଦେବା ,ଜ୍ବରର ନିରାକରଣ କରିବା ତଥା ସ୍ଥିର ପରିବେଶ ଦେବା ଇତ୍ୟାଦି ଗାଈର ଭୋକ ଓ ପାଚନ ଶକ୍ତି  ନ କମିଯିବା ର ନିରାକରଣ ଅଟେ।".split()
candidate = "ନମସ୍କାର 2ଡିଶା ସରକାରଙ୍କ ଆମକୃଷି ପ୍ରାଣୀଚିକିତ୍ସାଓ ପ୍ରାଣୀପାଳନ ସୂଚନା ସେବାକୁ ଆପଣଙ୍କୁ ସ୍ୱାଗତ। ଚାସି ବାହି ଓ ଭଣିମାନେ ଆଶନ୍ତୁ ଗାଈ ଓ ମଏସିମାନଙ୍କ କ୍ଷୁଧାହୀନତା ରୋଗର ନିରାକରଣ ବିଷୟରେ ଜାଣିବା ନିୟମିତ ସ୍ୱାସ୍ଥ୍ୟର ନିରୀକ୍ଷଣ ଟୀକାକରଣ ଓ କ୍ରମିନାସକ ଔଷଧର ପ୍ରୟୋଗ ପ୍ରୋବାୟୋଟିକ୍ସ ଓ ଯକୃତ ବର୍ଦ୍ଧକର ବ୍ୟବହାର ସଫାପାଣି ଯୋଗେଇବା ସୁଶମ ସନ୍ତୁଳିତ ଖାଦ୍ୟ ଇତ୍ୟାଦି ଦେବା ଋୋଷଇ ପରେ ବଳି ପଡ଼ୁଥିବା ଖାଦ୍ୟ ଇତ୍ୟାଦି ନ ଦେବା ପଶୁ ମାନଙ୍କ ଲାଗି ପ୍ରକୃଷ୍ଟ ଖାଦ୍ୟ ଇତ୍ୟାଦି ଖାଇବାକୁ ଦେବା ଫିମ୍ପିଯୁକ୍ତ ଖାଦ୍ୟ ନ ଦେବା ଜ୍ୱରର ନିରାକରଣ କରିବା ତଥା ସ୍ଥିର ପରିବେଶ ଦେବା ଇତ୍ୟାଦି ଗାଈର ଭୋକ ଓ ପାଚନ ଶକ୍ତି ନକମିଯିବାର ନିରାକରଣ ଅଟେ। ଏହି କୃଷୀ ସୂଚନା ଉପରେ ଅଧିକ ଜାଣିବା ପାଇଁ କିମ୍ବା କୃଷି ପଶୁ ପାଳନ ଓ ମଥ୍ୟଚାଷ ଉପରେ ଯେକୌଣସି ପ୍ରଶ୍ନ ପଚାରିବା ପାଇଁ ଆମ କୃଷୀର ନିଶୁଳ୍କ ନମ୍ବର 1155333 କୁ କଲ୍ କରନ୍ତୁ ଧନ୍ୟବାଦ।".split()

# Calculate BLEU score
bleu_score = sentence_bleu([reference], candidate)

# Print the BLEU score
print("BLEU Score:", bleu_score)


BLEU Score: 0.18066180177560492
